<a href="https://colab.research.google.com/github/YunTaeHyeon/AILab/blob/main/Titanic_testdataset_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [ ]:
#데이터 셋 업로드 및 확인
df_test = pd.read_csv('/content/test.csv')
df_submission=pd.read_csv('/content/gender_submission.csv')
#print(df_test.describe())

print()
print(df_test.isnull().sum().sum())


#df_test.loc[152]


414


In [ ]:
print(df_test.loc[152])

PassengerId                  1044
Pclass                          3
Name           Storey, Mr. Thomas
Sex                          male
Age                          60.5
SibSp                           0
Parch                           0
Ticket                       3701
Fare                          NaN
Cabin                         NaN
Embarked                        S
Name: 152, dtype: object


#필요없는 칼럼들 제거

In [ ]:
#필요없는 칼럼들 제거
df_test.drop(['Cabin','PassengerId'], axis=1, inplace=True)
df_test.dtypes

Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

# 성별 int형으로 변경



In [ ]:
#Sex데이터 남자 = 0, 여자는 1로 변환 및 데이터 형 변환
df_test.loc[df_test['Sex']=='male','Sex']=0
df_test.loc[df_test['Sex']=='female','Sex']=1
df_test = df_test.astype({'Sex':'int'})

df_test.dtypes

Pclass        int64
Name         object
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

# 호칭에 따른 계급 분류 Name -> Title



In [ ]:
#호칭에 따른 계급 분류(Name)

combine = [df_test]
title = df_test.Name.str.extract(' ([a-zA-Z]+)\.', expand=False)#영문자 뒤 .까지 추출
for df in combine:
  df['Title']=df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(df_test['Title'], df_test['Sex'])

Sex,0,1
Title,,
Col,2,0
Dona,0,1
Dr,1,0
Master,21,0
Miss,0,78
Mr,240,0
Mrs,0,72
Ms,0,1
Rev,2,0


In [ ]:
#많이 사용안되는 이름들 맵핑
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
#df_test[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
#분류된 이름들 숫자로 변환
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

df_test.head(12)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,Q,1
1,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,S,3
2,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,Q,1
3,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,S,1
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,S,3
5,3,"Svensson, Mr. Johan Cervin",0,14.0,0,0,7538,9.2250,S,1
6,3,"Connolly, Miss. Kate",1,30.0,0,0,330972,7.6292,Q,2
7,2,"Caldwell, Mr. Albert Francis",0,26.0,1,1,248738,29.0000,S,1
8,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",1,18.0,0,0,2657,7.2292,C,3
9,3,"Davies, Mr. John Samuel",0,21.0,2,0,A/4 48871,24.1500,S,1


In [ ]:
#사용하여 필요없어진 Name 컬럼 제거
df_test.drop(['Name'], axis=1, inplace=True)
df_test.dtypes

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
Title         int64
dtype: object

In [ ]:
#data_train.groupby(['Title',])['Survived'].mean().plot(kind="bar")
#data_train.groupby(['Title',])['Survived'].mean()*100


#Embarked 결측치 제거
: s가 가장 많기에 s를 삽입


In [ ]:
#Embarked 결측치 제거
print(df_test.groupby('Embarked').size())

df_test.Embarked = df_test.Embarked.fillna('S')
df_test.isnull().sum()

print(df_test.groupby('Embarked').size())

Embarked
C    102
Q     46
S    270
dtype: int64
Embarked
C    102
Q     46
S    270
dtype: int64


In [ ]:
#data_train.groupby(['Embarked',])['Survived'].mean().plot(kind="bar")
#data_train.groupby(['Embarked',])['Survived'].mean()*100

In [ ]:
combine = [df_test]
Embarked_mapping = {"C":0, "Q":1, "S":2}
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map(Embarked_mapping)
df_test.Embarked

0      1
1      2
2      1
3      2
4      2
      ..
413    2
414    0
415    2
416    2
417    0
Name: Embarked, Length: 418, dtype: int64

In [ ]:
df_test.dtypes

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked      int64
Title         int64
dtype: object

#Fare을 이용해 사람들을 분류


In [ ]:
print(df_test['Fare'].describe())
print(df_test['Fare'].mean())
df_test.dtypes

count    417.000000
mean      35.627188
std       55.907576
min        0.000000
25%        7.895800
50%       14.454200
75%       31.500000
max      512.329200
Name: Fare, dtype: float64
35.627188489208635


Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked      int64
Title         int64
dtype: object

In [ ]:
df_test.loc[df_test['Fare'].isnull() == True, 'Fare'] = df_test['Fare'].mean()

df_test.dtypes
print(df_test)

     Pclass  Sex   Age  SibSp  Parch              Ticket      Fare  Embarked  \
0         3    0  34.5      0      0              330911    7.8292         1   
1         3    1  47.0      1      0              363272    7.0000         2   
2         2    0  62.0      0      0              240276    9.6875         1   
3         3    0  27.0      0      0              315154    8.6625         2   
4         3    1  22.0      1      1             3101298   12.2875         2   
..      ...  ...   ...    ...    ...                 ...       ...       ...   
413       3    0   NaN      0      0           A.5. 3236    8.0500         2   
414       1    1  39.0      0      0            PC 17758  108.9000         0   
415       3    0  38.5      0      0  SOTON/O.Q. 3101262    7.2500         2   
416       3    0   NaN      0      0              359309    8.0500         2   
417       3    0   NaN      1      1                2668   22.3583         0   

     Title  
0        1  
1        3  


In [ ]:
#퍼센트별 인원수 정리 및 숫자로 맵핑
category = [-1, 7.910400, 14.454200, 31.000000, 512.329200]
name=[0,1,2,3]
df_test['Fare_level']=pd.cut(df_test['Fare'], category, labels= name)

df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,Fare_level
0,3,0,34.5,0,0,330911,7.8292,1,1,0
1,3,1,47.0,1,0,363272,7.0000,2,3,0
2,2,0,62.0,0,0,240276,9.6875,1,1,1
3,3,0,27.0,0,0,315154,8.6625,2,1,1
4,3,1,22.0,1,1,3101298,12.2875,2,3,1


In [ ]:
#data_train.groupby(['Fare_level',])['Survived'].mean().plot(kind="bar")
print(df_test.loc[df_test['Fare_level'].isnull() == True])
print(df_test.dtypes)

Empty DataFrame
Columns: [Pclass, Sex, Age, SibSp, Parch, Ticket, Fare, Embarked, Title, Fare_level]
Index: []
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked         int64
Title            int64
Fare_level    category
dtype: object


In [ ]:

df_test.drop(['Fare'], axis=1, inplace=True)
df_test = df_test.astype({'Fare_level':'int'})

In [ ]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Ticket,Embarked,Title,Fare_level
0,3,0,34.5,0,0,330911,1,1,0
1,3,1,47.0,1,0,363272,2,3,0
2,2,0,62.0,0,0,240276,1,1,1
3,3,0,27.0,0,0,315154,2,1,1
4,3,1,22.0,1,1,3101298,2,3,1
...,...,...,...,...,...,...,...,...,...
413,3,0,NaN,0,0,A.5. 3236,2,1,1
414,1,1,39.0,0,0,PC 17758,0,5,3
415,3,0,38.5,0,0,SOTON/O.Q. 3101262,2,1,0
416,3,0,NaN,0,0,359309,2,1,1


#TIcket을 이용한 사람 분류


In [ ]:
#STON/02. 과 STON/0 2. 은 가격과 pclass에서 같은 모습을 보이기에 같은 분류로 판단 

df_test['Ticket'] = df_test['Ticket'].str.replace('STON/O 2.','STON/O2.')

<ipython-input-118-3047d05363b4>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Ticket'] = df_test['Ticket'].str.replace('STON/O 2.','STON/O2.')


In [ ]:
df_test['Ticket'].value_counts()

ticket_level = df_test['Ticket'].to_list()

In [ ]:
count={}
for i in ticket_level:
  try: count[i] +=1
  except: count[i]=1

d2 = sorted(count.items(), key=lambda x: x[1], reverse=True)

In [ ]:
combine = [df_test] 
for dataset in combine:
    dataset['Ticket'] = dataset['Ticket'].map(count)
    

print(df_test['Ticket'])
#df_test.groupby(['Ticket',])['Survived'].mean().plot(kind="bar")

0      1
1      1
2      1
3      1
4      1
      ..
413    1
414    1
415    1
416    1
417    1
Name: Ticket, Length: 418, dtype: int64


In [ ]:
df_test.dtypes

Pclass          int64
Sex             int64
Age           float64
SibSp           int64
Parch           int64
Ticket          int64
Embarked        int64
Title           int64
Fare_level      int64
dtype: object

#동승자 유무에 따른 생존률 분석

In [ ]:
#data=pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
#df= pd.DataFrame(data)

#df['Passenger'] = df['SibSp'] + df['Parch']


In [ ]:
#s = df[df['Survived']==1]

#p1 = pd.merge(df[df['Passenger']==0], s)
#p2 = pd.merge(df[df['Passenger']!=0], s)

#a = p1.size/df[df['Passenger'] == 0].size
#b = p2.size/df[df['Passenger'] != 0].size

In [ ]:
#x = np.arange(2)
#xvalue=[0,1]
#yvalue=[a,b]

#plt.bar(x, yvalue)
#plt.xticks(x, xvalue)

#plt.show()

#동승자가 있는 사람의 생존 확률이 더 높음

In [ ]:
#df_test['Passenger']= df_test['SibSp'] + df_test['Parch']

#df_test.loc[df_test['Passenger']==0,'Fellow']=0
#df_test.loc[df_test['Passenger']!=0,'Fellow']=1

In [ ]:
#df_test = df_test.astype({'Fellow':'int'})
#df_test['Fellow']
#df_test.drop(['Passenger','SibSp','Parch'], axis=1, inplace=True)

In [ ]:
#df_test.dtypes

#나이에 따른 분류

In [ ]:

#count = 0
#sum1=0

#a1=data[data['Age'] <= 10]
#a2=data[data['Age'] <= 20]
#a3=data[data['Age'] <= 30]
#a4=data[data['Age'] <= 40]
#a5=data[data['Age'] <= 50]
#a6=data[data['Age'] <= 60]
#a7=data[data['Age'] <= 70]
#a8=data[data['Age'] <= 80]

#a8=pd.merge(a8,a7,how='outer')
#a7=pd.merge(a7,a6,how='outer')
#a6=pd.merge(a6,a5,how='outer')
#a5=pd.merge(a5,a4,how='outer')
#a4=pd.merge(a4,a3,how='outer')
#a3=pd.merge(a3,a2,how='outer')
#a2=pd.merge(a2,a1,how='outer')

#b=data[data['Survived']==1]

#d1=pd.merge(a1,b)
#d2=pd.merge(a2,b)
#d3=pd.merge(a3,b)
#d4=pd.merge(a4,b)
#d5=pd.merge(a5,b)
#d6=pd.merge(a6,b)
#d7=pd.merge(a7,b)
#d8=pd.merge(a8,b)


#x=np.arange(8)
#xvalue=[1,2,3,4,5,6,7,8]
#yvalue=[len(d1)/len(a1),len(d2)/len(a2),len(d3)/len(a3),len(d4)/len(a4),
#        len(d5)/len(a5),len(d6)/len(a6),len(d7)/len(a7),len(d8)/len(a8)]

#plt.bar(x, yvalue)
#plt.xticks(x, xvalue)

#plt.show()

In [ ]:
#age에 있는 결측치 채우기
print(df_test['Age'].median())
df_test.loc[df_test['Age'].isnull(),'Age']=df_test['Age'].median()

df_test

27.0


,Pclass,Sex,Age,SibSp,Parch,Ticket,Embarked,Title,Fare_level
0,3,0,34.5,0,0,1,1,1,0
1,3,1,47.0,1,0,1,2,3,0
2,2,0,62.0,0,0,1,1,1,1
3,3,0,27.0,0,0,1,2,1,1
4,3,1,22.0,1,1,1,2,3,1
...,...,...,...,...,...,...,...,...,...
413,3,0,27.0,0,0,1,2,1,1
414,1,1,39.0,0,0,1,0,5,3
415,3,0,38.5,0,0,1,2,1,0
416,3,0,27.0,0,0,1,2,1,1


In [ ]:
#10살 단위로 분류
#df_test.loc[df_test['Age']<=80,'Age_class']=7
#df_test.loc[df_test['Age']<=70,'Age_class']=6
#df_test.loc[df_test['Age']<=60,'Age_class']=5
#df_test.loc[df_test['Age']<=50,'Age_class']=4
#df_test.loc[df_test['Age']<=40,'Age_class']=3
#df_test.loc[df_test['Age']<=30,'Age_class']=2
#df_test.loc[df_test['Age']<=20,'Age_class']=1
#df_test.loc[df_test['Age']<=10,'Age_class']=0

#df_test.loc[152]

In [ ]:
#df_test = df_test.astype({'Age_class':'int'})

#df_test.drop(['Age'], axis=1, inplace=True)

#print(df_test)

In [ ]:
df_test.dtypes
print(df_test)

     Pclass  Sex   Age  SibSp  Parch  Ticket  Embarked  Title  Fare_level
0         3    0  34.5      0      0       1         1      1           0
1         3    1  47.0      1      0       1         2      3           0
2         2    0  62.0      0      0       1         1      1           1
3         3    0  27.0      0      0       1         2      1           1
4         3    1  22.0      1      1       1         2      3           1
..      ...  ...   ...    ...    ...     ...       ...    ...         ...
413       3    0  27.0      0      0       1         2      1           1
414       1    1  39.0      0      0       1         0      5           3
415       3    0  38.5      0      0       1         2      1           0
416       3    0  27.0      0      0       1         2      1           1
417       3    0  27.0      1      1       1         0      4           2

[418 rows x 9 columns]


In [ ]:
df_test = df_test.astype({'Pclass':'int'})
df_test = df_test.astype({'Sex':'int'})
df_test = df_test.astype({'Embarked':'int'})
df_test = df_test.astype({'Title':'int'})
df_test.dtypes

Pclass          int64
Sex             int64
Age           float64
SibSp           int64
Parch           int64
Ticket          int64
Embarked        int64
Title           int64
Fare_level      int64
dtype: object

In [ ]:
df_test=df_test['PassengerId','Survived']

KeyError: ignored

In [ ]:
df_test.to_csv('/content/drive/MyDrive/test_pre.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(df_test['Age'].isnull().sum())